In [ ]:
import pandas as pd
import math
import numpy as np
from geopy.distance import distance

# Lire le fichier CSV dans un DataFrame
try:
    df = pd.read_csv('drones.csv')
except FileNotFoundError:
    print("Le fichier 'drones_original.csv' est introuvable. Veuillez vérifier le chemin du fichier.")
    exit()

# Liste des colonnes à supprimer
columns_to_drop = []  # Remplacez par les noms réels des colonnes à supprimer
# Supprimer les colonnes
df = df.drop(columns=columns_to_drop)

# Dictionnaire des coordonnées des stations avec élévation
stations_coordinates = {
    '0QRDJCAR037P0N': {'name': 'Luxembourg', 'latitude': 49.624918, 'longitude': 6.150544, 'elevation': 353, 'altitude': 0}, 
    '0QRDJCAR0383TD': {'name': 'Paris', 'latitude': 49.010125, 'longitude': 2.566628, 'elevation': 111, 'altitude': 0}, 
    '0QRDKC2R038370': {'name': 'Vienne', 'latitude': 48.10424, 'longitude': 16.589581, 'elevation': 183, 'altitude': 0}, 
    '0QRDKC2R03J32P': {'name': 'Cyprus', 'latitude': 34.876573, 'longitude': 33.631523, 'elevation': 2, 'altitude': 0}, 
    '0QRDE8F0010496': {'name': 'London', 'latitude': 51.517741, 'longitude': -0.059022, 'elevation': 100, 'altitude': 0},
    'F5FHD235C00D05SS': {'name': 'Brussels', 'latitude': 50.883293, 'longitude': 4.428564, 'elevation': 55, 'altitude': 0},
    '11VKJ5H00201YG': {'name': 'Bordeaux', 'latitude': 44.831398, 'longitude': -0.702665, 'elevation': 51, 'altitude': 0},
    'F62HD227X00CL245': {'name': 'Belfast', 'latitude': 54.613344, 'longitude': -5.87362, 'elevation': 4, 'altitude': 0},
}

# Ajouter une colonne pour la ville attribuée
df['city'] = None

# Fonction pour calculer la distance entre deux points géographiques
def calculate_distance(lat1, lon1, lat2, lon2):
    coords_1 = (lat1, lon1)
    coords_2 = (lat2, lon2)
    return distance(coords_1, coords_2).km

# Itérer sur chaque station pour attribuer une ville aux drones dans un rayon de 100 km
for station_id, station_info in stations_coordinates.items():
    station_lat = station_info['latitude']
    station_lon = station_info['longitude']
    city_name = station_info['name']
    
    # Filtrer les drones dans un rayon de 100 km autour de la station
    df_station = df[df.apply(lambda row: calculate_distance(row['latitude'], row['longitude'], station_lat, station_lon), axis=1) <= 100]
    
    # Assigner la ville à ces drones
    df.loc[df_station.index, 'city'] = city_name

# Fonction pour convertir les coordonnées géographiques en coordonnées cartésiennes
def geo_to_cartesian(row):
    R = 6371  # Rayon moyen de la Terre en kilomètres
    
    # Convertir les coordonnées géographiques en radians
    lat_rad = np.radians(row['latitude'])
    lon_rad = np.radians(row['longitude'])
    alt = row['altitude']
    
    # Calculer les coordonnées cartésiennes
    x = (R + alt) * np.cos(lat_rad) * np.cos(lon_rad)
    y = (R + alt) * np.cos(lat_rad) * np.sin(lon_rad)
    z = (R + alt) * np.sin(lat_rad)
    
    return pd.Series([x, y, z])

# Appliquer la fonction de conversion aux lignes du DataFrame
df[['x', 'y', 'z']] = df.apply(geo_to_cartesian, axis=1)

# Supprimer les lignes où la colonne 'city' contient des NaN
df_cleaned = df.dropna(subset=['city'])

# Exporter le DataFrame nettoyé vers un nouveau fichier CSV
output_file = 'drones_cleaned1.csv'
df_cleaned.to_csv(output_file, index=False)

print(f"Les lignes avec NaN dans la colonne 'city' ont été supprimées.")
print(f"Le DataFrame nettoyé a été exporté vers '{output_file}'.")


In [ ]:
import pandas as pd
import numpy as np

# Chargement des données à partir d'un fichier CSV
df = pd.read_csv('drones_converted.csv')

# Arrondir les valeurs de la colonne 'angle_from_station' à l'unité la plus proche
df['angle_from_station'] = df['angle_from_station'].round()

# Convertir 360 en 0
df['angle_from_station'] = df['angle_from_station'].apply(lambda x: 0 if x == 360 else x)

# Réécriture des données dans le fichier CSV
df.to_csv('drones_converted_rounded.csv', index=False)

print("Les angles ont été arrondis et enregistrés dans 'drones_converted_rounded.csv'.")



import pandas as pd
import numpy as np

# Chargement des données à partir du fichier CSV
df = pd.read_csv('drones_converted_rounded.csv')


# Map city names to station IDs
city_to_station_id = {value['name']: key for key, value in stations_coordinates.items()}

# Calcul de l'angle vertical pour chaque drone
def calculate_vertical_angle(row):
    # Get station elevation
    station_id = city_to_station_id.get(row['city'])
    if not station_id:
        return np.nan  # Return NaN if station ID not found
    
    station_elevation = stations_coordinates[station_id]['elevation']
    # Calcul de la différence d'altitude
    height_difference = row['altitude'] - station_elevation
    # Calcul de l'angle en radians puis conversion en degrés
    angle_rad = np.arctan2(height_difference, row['distance_haversine'])
    angle_deg = np.degrees(angle_rad)
    return angle_deg



# Calcul du nombre de données par angle par ville
data_counts = df.groupby(['city', 'angle_from_station']).size().reset_index(name='counts')


# Ajouter une colonne pour l'angle vertical dans le DataFrame
df['vertical_angle'] = df.apply(calculate_vertical_angle, axis=1)

# Arrondir les valeurs de la colonne 'vertical_angle' à l'unité la plus proche
df['vertical_angle'] = df['vertical_angle'].round()

# Réécriture des données dans le fichier CSV
df.to_csv('drones_converted_with_vertical_angles.csv', index=False)

print("Les angles verticaux ont été calculés, arrondis et enregistrés dans 'drones_converted_with_vertical_angles.csv'.")



In [78]:
import pandas as pd

# Charger les données sur les drones
drones_df = pd.read_csv('drones_mis_a_jour.csv')

# Convertir les timestamps Unix en format YY_MM_DD HH:MM:SS
drones_df['timestamp_readable'] = pd.to_datetime(drones_df['time'], unit='s').dt.strftime('%Y_%m_%d %H:%M:%S')

# Afficher les premières lignes pour vérifier
print(drones_df.head())

# Enregistrer le DataFrame mis à jour
drones_df.to_csv('drones_mis_a_jour.csv', index=False)


    latitude  longitude  altitude    station_name  station_latitude  \
0  48.622917   2.245473     177.0  0QRDE8F0010496         48.600016   
1  48.622917   2.245473     177.0  0QRDE8F0010496         48.600016   
2  48.622912   2.245467     177.0  0QRDE8F0010496         48.600016   
3  48.622912   2.245462     174.0  0QRDE8F0010496         48.600016   
4  48.622912   2.245462     194.0  0QRDE8F0010496         48.600016   

   station_longitude        time  angle_from_station  vertical_angle  \
0           2.347032  1631346230               167.0            83.0   
1           2.347025  1631346234               167.0            83.0   
2           2.347025  1631346236               167.0            83.0   
3           2.347022  1631346240               167.0            83.0   
4           2.347014  1631346254               167.0            85.0   

    timestamp_readable  
0  2021_09_11 07:43:50  
1  2021_09_11 07:43:54  
2  2021_09_11 07:43:56  
3  2021_09_11 07:44:00  
4  2021_09_11 0

In [84]:
import pandas as pd

# Charger les données sur les antennes
antennas_data = pd.read_csv('ant.csv')
# Charger le fichier CSV des enregistrements de drones
drone_records = pd.read_csv('drones_mis_a_jour.csv')



# Fonction pour identifier l'emplacement de l'antenne à un moment donné
def identify_antenna_location(row):
    station_name = row['station_name']
    timestamp = row['timestamp_readable']

    # Filtrer les installations pour le NAME_NUM spécifique
    filtered_installations = antennas_data[antennas_data['NAME_NUM'] == station_name]
    
    # Trouver l'emplacement correspondant à la date/heure du timestamp
    for index, installation in filtered_installations.iterrows():
        start_at = installation['START_AT']
        end_at = installation['END_AT'] if pd.notna(installation['END_AT']) else pd.Timestamp.now()
        
        if start_at <= timestamp <= end_at:
            return installation['NAME']
    
    return None

# Appliquer la fonction pour identifier l'emplacement sur chaque ligne du dataframe des drones
drone_records['ville'] = drone_records.apply(identify_antenna_location, axis=1)

# Afficher les premières lignes pour vérifier
print(drone_records.head())

# Exporter le dataframe mis à jour avec la nouvelle colonne
drone_records.to_csv('drones_with_locations.csv', index=False)

print("Fichier exporté avec succès avec la colonne identifiant l'emplacement des antennes.")


    latitude  longitude  altitude    station_name  station_latitude  \
0  48.622917   2.245473     177.0  0QRDE8F0010496         48.600016   
1  48.622917   2.245473     177.0  0QRDE8F0010496         48.600016   
2  48.622912   2.245467     177.0  0QRDE8F0010496         48.600016   
3  48.622912   2.245462     174.0  0QRDE8F0010496         48.600016   
4  48.622912   2.245462     194.0  0QRDE8F0010496         48.600016   

   station_longitude        time  angle_from_station  vertical_angle  \
0           2.347032  1631346230               167.0            83.0   
1           2.347025  1631346234               167.0            83.0   
2           2.347025  1631346236               167.0            83.0   
3           2.347022  1631346240               167.0            83.0   
4           2.347014  1631346254               167.0            85.0   

   timestamp_readable ville  
0 2021-09-11 07:43:50   EIH  
1 2021-09-11 07:43:54   EIH  
2 2021-09-11 07:43:56   EIH  
3 2021-09-11 07:44:0

In [76]:
import pandas as pd

# Charger les données sur les antennes
antennes_df = pd.read_csv('ant.csv')

# Convertir les colonnes START_AT et END_AT en objets Timestamp
antennes_df['START_AT'] = pd.to_datetime(antennes_df['START_AT'])
antennes_df['END_AT'] = pd.to_datetime(antennes_df['END_AT'], errors='coerce')  # 'coerce' pour gérer les valeurs non valides

# Charger les données sur les drones
drones_df = pd.read_csv('drones_mis_a_jour.csv')

# Ajouter une colonne pour le nom de la ville
drones_df['ville'] = ''

# Parcourir les drones et associer le nom de la ville en fonction du timestamp
for index, row in drones_df.iterrows():
    timestamp_str = row['timestamp_readable']
    station_name = row['station_name']
    
    # Convertir le timestamp_readable en Timestamp
    timestamp = pd.to_datetime(timestamp_str, format='%Y_%m_%d %H:%M:%S')
    
    # Rechercher les antennes correspondantes
    matching_antennas = antennes_df[(antennes_df['NAME_NUM'] == station_name) & 
                                    (antennes_df['START_AT'] <= timestamp) & 
                                    ((antennes_df['END_AT'].isnull()) | (antennes_df['END_AT'] >= timestamp))]
    
    if not matching_antennas.empty:
        # Sélectionner la première antenne correspondante (peut-être optimisable selon vos besoins)
        antenna_row = matching_antennas.iloc[0]
        ville = antenna_row['NAME']
        drones_df.at[index, 'ville'] = ville
        

# Enregistrer le dataframe mis à jour
drones_df.to_csv('drones_mis_a_jour.csv', index=False)


KeyError: 'timestamp_readable'

In [44]:
import pandas as pd

# Charger les données mises à jour sur les drones
drones_df = pd.read_csv('drones_mis_a_jour.csv')

# Compter le nombre de lignes où la colonne 'ville' est NaN
missing_ville_count = drones_df['ville'].isna().sum()

print(f"Le nombre de lignes sans ville associée est : {missing_ville_count}")


Le nombre de lignes sans ville associée est : 55124


In [75]:
import pandas as pd


# Lire le fichier CSV dans un DataFrame
try:
    df = pd.read_csv('drones_converted_with_vertical_angles.csv')
except FileNotFoundError:
    print("Le fichier 'drones_original.csv' est introuvable. Veuillez vérifier le chemin du fichier.")
    exit()

# Liste des colonnes à supprimer
columns_to_drop = ['journey','ident','model','heading','year','month','day','hour','minute','second','distance_haversine','city','station_elevation']  # Remplacez par les noms réels des colonnes à supprimer
# Supprimer les colonnes
df = df.drop(columns=columns_to_drop)


# Enregistrer le dataframe mis à jour
df.to_csv('drones_mis_a_jour.csv', index=False)

In [88]:
import pandas as pd

# Lire les fichiers CSV
file_df = pd.read_csv('modified_file_with_distances.csv')
ant_df = pd.read_csv('ant.csv')

# Supprimer les doublons en gardant uniquement la première occurrence de chaque NAME_NUM
ant_df = ant_df.drop_duplicates(subset='NAME_NUM')

# Renommer les colonnes pour préparer la fusion
ant_df = ant_df.rename(columns={'NAME_NUM': 'station_name', 'LATITUDE': 'station_latitude', 'LONGITUDE': 'station_longitude', 'REF_ALTITUDE': 'station_elevation'})

# Fusionner les DataFrames sur la colonne 'station_name'
merged_df = file_df.merge(ant_df[['station_name', 'station_latitude', 'station_longitude', 'station_elevation']], on='station_name', how='left')

# Mettre à jour les colonnes existantes et ajouter la nouvelle colonne
merged_df['station_latitude'] = merged_df['station_latitude_y']
merged_df['station_longitude'] = merged_df['station_longitude_y']
merged_df['station_elevation'] = merged_df['station_elevation']

# Supprimer les colonnes temporaires
merged_df = merged_df.drop(columns=['station_latitude_y', 'station_longitude_y'])

# Sauvegarder le fichier modifié
merged_df.to_csv('modified_file_with_elevation.csv', index=False)


In [90]:
import pandas as pd

# Lire le fichier CSV
file_df = pd.read_csv('modified_file_with_elevation.csv')

# Soustraire station_elevation de altitude
file_df['true_altitude'] = file_df['altitude'] - file_df['station_elevation']


# Sauvegarder le fichier modifié
file_df.to_csv('modified_file_with_real_alt.csv', index=False)


In [87]:
import pandas as pd
from math import radians, sin, cos, sqrt, atan2





# Fonction pour calculer la distance haversine entre deux points géographiques
def haversine(lat1, lon1, lat2, lon2):
    R = 6371.0  # Rayon de la Terre en km

    # Convertir les coordonnées degrés décimaux en radians
    lat1_rad = radians(lat1)
    lon1_rad = radians(lon1)
    lat2_rad = radians(lat2)
    lon2_rad = radians(lon2)

    # Calculer les différences de latitude et de longitude
    dlon = lon2_rad - lon1_rad
    dlat = lat2_rad - lat1_rad

    # Calculer la distance haversine
    a = sin(dlat / 2)**2 + cos(lat1_rad) * cos(lat2_rad) * sin(dlon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))
    distance = R * c

    return distance

# Charger le fichier CSV des drones
file_df = pd.read_csv('drones_with_locations.csv')


# Liste pour stocker les distances calculées
distances = []

# Calculer la distance haversine pour chaque ligne du DataFrame
for index, row in file_df.iterrows():
    # Coordonnées du drone
    drone_lat = row['latitude']
    drone_lon = row['longitude']

    # Définir les coordonnées de la station (hypothétique)
    station_lat =row['station_latitude']
    station_lon = row['station_longitude']

    
    # Calculer la distance haversine entre le drone et la station
    distance = haversine(station_lat, station_lon, drone_lat, drone_lon)
    distances.append(distance)

# Ajouter la liste de distances calculées au DataFrame
file_df['distance_to_station'] = distances

# Sauvegarder le fichier modifié avec les nouvelles colonnes
file_df.to_csv('modified_file_with_distances.csv', index=False)

print("Fichier CSV modifié avec succès.")


Fichier CSV modifié avec succès.


In [71]:
import pandas as pd
import numpy as np

# Charger le fichier CSV modifié avec les angles
df = pd.read_csv('modified_file_with_angles.csv')

# Fonction pour arrondir les angles
def round_angles(angle):
    if angle >= 0 and angle < 360:
        rounded_angle = int(np.round(angle))
        if rounded_angle == 360:
            rounded_angle = 0
        return rounded_angle
    else:
        return np.nan  # Gérer les valeurs non valides si nécessaire

# Appliquer la fonction d'arrondi aux colonnes d'angles
df['horizontal_angle_to_station'] = df['horizontal_angle_to_station'].apply(round_angles)
df['vertical_angle_to_station'] = df['vertical_angle_to_station'].apply(round_angles)

# Sauvegarder le fichier CSV modifié avec les angles arrondis
df.to_csv('modified_file_with_rounded_angles.csv', index=False)

print("Angles arrondis avec succès.")


KeyError: 'horizontal_angle_to_station'

In [2]:
import pandas as pd
import numpy as np

# Charger le fichier CSV modifié avec les angles
df = pd.read_csv('../CSV/modified_file_with_real_alt.csv')

display(df)

,latitude,longitude,altitude,station_name,station_latitude_x,station_longitude_x,time,angle_from_station,vertical_angle,timestamp_readable,ville,distance_to_station,station_elevation,station_latitude,station_longitude,true_altitude
0,48.622917,2.245473,177.0,0QRDE8F0010496,48.600016,2.347032,1631346230,167.0,83.0,2021-09-11 07:43:50,EIH,7.888702,89.0,48.6,2.35,88.0
1,48.622917,2.245473,177.0,0QRDE8F0010496,48.600016,2.347025,1631346234,167.0,83.0,2021-09-11 07:43:54,EIH,7.888214,89.0,48.6,2.35,88.0
2,48.622912,2.245467,177.0,0QRDE8F0010496,48.600016,2.347025,1631346236,167.0,83.0,2021-09-11 07:43:56,EIH,7.888453,89.0,48.6,2.35,88.0
3,48.622912,2.245462,174.0,0QRDE8F0010496,48.600016,2.347022,1631346240,167.0,83.0,2021-09-11 07:44:00,EIH,7.888592,89.0,48.6,2.35,85.0
4,48.622912,2.245462,194.0,0QRDE8F0010496,48.600016,2.347014,1631346254,167.0,85.0,2021-09-11 07:44:14,EIH,7.888035,89.0,48.6,2.35,105.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
744129,34.918626,33.624613,69.0,0QRDKC2R03J32P,34.876518,33.631536,1714021620,99.0,86.0,2024-04-25 05:07:00,CYP,4.724573,56.0,50.8,4.40,13.0
744130,34.919045,33.624613,69.0,0QRDKC2R03J32P,34.876515,33.631536,1714021624,99.0,86.0,2024-04-25 05:07:04,CYP,4.771080,56.0,50.8,4.40,13.0
744131,34.919663,33.623077,68.0,0QRDKC2R03J32P,34.876512,33.631530,1714021640,101.0,86.0,2024-04-25 05:07:20,CYP,4.859707,56.0,50.8,4.40,12.0
744132,34.918483,33.624361,52.0,0QRDKC2R03J32P,34.876502,33.631530,1714021752,100.0,85.0,2024-04-25 05:09:12,CYP,4.713638,56.0,50.8,4.40,-4.0


In [3]:
import pandas as pd
import os

# Chargement des données de station
station_data = """
ID,NAME,CODE,LATITUDE,LONGITUDE,REF_ALTITUDE
1,"EIH","8FW4H8XX+XX",48.6,2.35,89
2,"CDG","8FX42H52+X2",49.01,2.55,119
3,"LUX","8FX8H5XX+XX",49.6,6.2,376
4,"BRU","9F26RC22+22",50.8,4.4,56
5,"BEL","9C6MMRX2+X2",54.7,-6.2,82
6,"BDX","8CPXR822+22",44.8,-0.7,49
7,"LON","9C3X5WXX+XX",51.52,-0.05,6
8,"CYP","8G7M575J+X2",35.16,33.28,3
9,"BUC","8GP8H39X+XX",44.57,26.1,96
10,"AUS","8FWR3HXR+X2",48.10,16.59,183
"""

from io import StringIO
station_df = pd.read_csv(StringIO(station_data))

# Affichage des données de station
print("Stations data:")
print(station_df)

# Chargement des données de drones
drone_df = pd.read_csv('../CSV/modified_file_with_real_alt.csv')

# Affichage des données de drones
print("\nDrone data:")
print(drone_df)

# Ajout des données d'altitude des stations au DataFrame des drones
merged_df = pd.merge(drone_df, station_df[['NAME', 'REF_ALTITUDE']], left_on='ville', right_on='NAME', how='left')

# Recalcul de l'altitude réelle (real_altitude)
merged_df['real_altitude'] = merged_df['altitude'] - merged_df['REF_ALTITUDE']

# Affichage des résultats
print("\nMerged data with recalculated real_altitude:")
print(merged_df[['latitude', 'longitude', 'altitude', 'station_name', 'station_latitude_x', 'station_longitude_x', 'time', 'angle_from_station', 'vertical_angle', 'timestamp_readable', 'ville', 'distance_to_station', 'REF_ALTITUDE', 'real_altitude']])

# Sauvegarde du DataFrame modifié dans un fichier CSV

df.to_csv('modified_file_with_real_alt_recalculated.csv' , index=False)

print('fini')

Stations data:
   ID NAME         CODE  LATITUDE  LONGITUDE  REF_ALTITUDE
0   1  EIH  8FW4H8XX+XX     48.60       2.35            89
1   2  CDG  8FX42H52+X2     49.01       2.55           119
2   3  LUX  8FX8H5XX+XX     49.60       6.20           376
3   4  BRU  9F26RC22+22     50.80       4.40            56
4   5  BEL  9C6MMRX2+X2     54.70      -6.20            82
5   6  BDX  8CPXR822+22     44.80      -0.70            49
6   7  LON  9C3X5WXX+XX     51.52      -0.05             6
7   8  CYP  8G7M575J+X2     35.16      33.28             3
8   9  BUC  8GP8H39X+XX     44.57      26.10            96
9  10  AUS  8FWR3HXR+X2     48.10      16.59           183

Drone data:
         latitude  longitude  altitude    station_name  station_latitude_x  \
0       48.622917   2.245473     177.0  0QRDE8F0010496           48.600016   
1       48.622917   2.245473     177.0  0QRDE8F0010496           48.600016   
2       48.622912   2.245467     177.0  0QRDE8F0010496           48.600016   
3       48.

In [4]:
import pandas as pd
import os
from io import StringIO

# Chargement des données de station
station_data = """
ID,NAME,CODE,LATITUDE,LONGITUDE,REF_ALTITUDE
1,"EIH","8FW4H8XX+XX",48.6,2.35,89
2,"CDG","8FX42H52+X2",49.01,2.55,119
3,"LUX","8FX8H5XX+XX",49.6,6.2,376
4,"BRU","9F26RC22+22",50.8,4.4,56
5,"BEL","9C6MMRX2+X2",54.7,-6.2,82
6,"BDX","8CPXR822+22",44.8,-0.7,49
7,"LON","9C3X5WXX+XX",51.52,-0.05,6
8,"CYP","8G7M575J+X2",35.16,33.28,3
9,"BUC","8GP8H39X+XX",44.57,26.1,96
10,"AUS","8FWR3HXR+X2",48.10,16.59,183
"""

station_df = pd.read_csv(StringIO(station_data))

# Affichage des données de station
print("Stations data:")
print(station_df)

# Chargement des données de drones
drone_df = pd.read_csv('../CSV/modified_file_with_real_alt.csv')

# Affichage des données de drones
print("\nDrone data:")
print(drone_df)

# Ajout des données d'altitude des stations au DataFrame des drones
merged_df = pd.merge(drone_df, station_df[['NAME', 'REF_ALTITUDE']], left_on='ville', right_on='NAME', how='left')

# Recalcul de l'altitude réelle (real_altitude)
merged_df['real_altitude'] = merged_df['altitude'] - merged_df['REF_ALTITUDE']

# Affichage des résultats
print("\nMerged data with recalculated real_altitude:")
print(merged_df[['latitude', 'longitude', 'altitude', 'station_name', 'station_latitude_x', 'station_longitude_x', 'time', 'angle_from_station', 'vertical_angle', 'timestamp_readable', 'ville', 'distance_to_station', 'REF_ALTITUDE', 'real_altitude']])

# Sauvegarde du DataFrame modifié dans un fichier CSV
merged_df.to_csv('../CSV/modified_file_with_real_alt_recalculated.csv', index=False)

print('fini')


Stations data:
   ID NAME         CODE  LATITUDE  LONGITUDE  REF_ALTITUDE
0   1  EIH  8FW4H8XX+XX     48.60       2.35            89
1   2  CDG  8FX42H52+X2     49.01       2.55           119
2   3  LUX  8FX8H5XX+XX     49.60       6.20           376
3   4  BRU  9F26RC22+22     50.80       4.40            56
4   5  BEL  9C6MMRX2+X2     54.70      -6.20            82
5   6  BDX  8CPXR822+22     44.80      -0.70            49
6   7  LON  9C3X5WXX+XX     51.52      -0.05             6
7   8  CYP  8G7M575J+X2     35.16      33.28             3
8   9  BUC  8GP8H39X+XX     44.57      26.10            96
9  10  AUS  8FWR3HXR+X2     48.10      16.59           183

Drone data:
         latitude  longitude  altitude    station_name  station_latitude_x  \
0       48.622917   2.245473     177.0  0QRDE8F0010496           48.600016   
1       48.622917   2.245473     177.0  0QRDE8F0010496           48.600016   
2       48.622912   2.245467     177.0  0QRDE8F0010496           48.600016   
3       48.

In [6]:
import pandas as pd
import numpy as np

# Charger les données
df = pd.read_csv('../CSV/modified_file_with_real_alt_recalculated.csv')

# Calculer la différence d'altitude
df['altitude_difference'] = df['altitude'] - df['station_elevation']

# Calculer l'angle vertical en radians
df['vertical_angle_radians'] = np.arctan2(df['altitude_difference'], df['distance_to_station'])

# Convertir l'angle vertical en degrés
df['vertical_angle_degrees'] = np.degrees(df['vertical_angle_radians'])

# Arrondir les angles verticaux à l'unité
df['vertical_angle_degrees'] = df['vertical_angle_degrees'].round()

# Limiter les angles entre -90 et 90 degrés
df['vertical_angle_degrees'] = np.clip(df['vertical_angle_degrees'], -90, 90)

# Afficher les résultats
print(df[['latitude', 'longitude', 'altitude', 'station_name', 'distance_to_station', 'station_elevation', 'altitude_difference', 'vertical_angle_degrees']])

# Exporter le DataFrame modifié dans un nouveau fichier CSV
output_file = '../CSV/modified_file_with_real_alt_recalculated_and_vertical_angle.csv'
df.to_csv(output_file, index=False)

print(f'Fichier CSV exporté avec succès : {output_file}')


         latitude  longitude  altitude    station_name  distance_to_station  \
0       48.622917   2.245473     177.0  0QRDE8F0010496             7.888702   
1       48.622917   2.245473     177.0  0QRDE8F0010496             7.888214   
2       48.622912   2.245467     177.0  0QRDE8F0010496             7.888453   
3       48.622912   2.245462     174.0  0QRDE8F0010496             7.888592   
4       48.622912   2.245462     194.0  0QRDE8F0010496             7.888035   
...           ...        ...       ...             ...                  ...   
744129  34.918626  33.624613      69.0  0QRDKC2R03J32P             4.724573   
744130  34.919045  33.624613      69.0  0QRDKC2R03J32P             4.771080   
744131  34.919663  33.623077      68.0  0QRDKC2R03J32P             4.859707   
744132  34.918483  33.624361      52.0  0QRDKC2R03J32P             4.713638   
744133  34.918626  33.623788      46.0  0QRDKC2R03J32P             4.735875   

        station_elevation  altitude_difference  ver